In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import lxml.html
import urllib
import bleach
import re


ModuleNotFoundError: No module named 'wordcloud'

In [2]:
# function that takes in url and returns author and body
def get_author_body(url):
    full_url = "https://www.churchofjesuschrist.org" + url
    mysite = urllib.request.urlopen(full_url).read()

    soup = BeautifulSoup(mysite)
    body_block = soup.find_all(class_="body-block")[0]
    try:
        author_html = soup.find_all(class_="author-name")[0]
    except:
        try:
            author_html = soup.find(id='p1')
        except:
            print(full_url)

    # clean author's name from html
    author_text = bleach.clean(author_html.text, tags=[], strip=True)
    author = author_text.replace('By ', '')

    # remove all reference numbers from text
    for ref in body_block("a"):
        ref.decompose()

    return author, re.search(r"[\d]{4}/[\d]{2}", url)[0], body_block.text


# this function takes in a url and gets the links for all the conference talks
def get_talk_urls_from_conf(url): 
    full_url = "https://www.churchofjesuschrist.org" + url
    r = requests.get(full_url)
    if r.status_code == 200:
        r_soup = BeautifulSoup(r.text)
        talks = r_soup.find_all(class_="lumen-tile__link", href=True)
        urls = []
        for talk in talks:
            urls.append(talk['href'])
        return(urls)


# retuns list of conference urls
def get_all_conf_urls(url):
    r = requests.get(all_conf_url)
    if r.status_code == 200:
        r_soup = BeautifulSoup(r.text)
        conferences = r_soup.find_all(class_="year-line__link", href=True)
        urls = []
        for conference in conferences:
            urls.append(conference['href'])
        return urls

            
        

In [3]:
all_conf_url = "https://www.churchofjesuschrist.org/general-conference/conferences?lang=eng"
conf_urls = get_all_conf_urls(all_conf_url)

# conf_url_04_2020 = "https://www.churchofjesuschrist.org/general-conference/2020/04?lang=eng"

all_talks_urls = []

for url in conf_urls[:-1]:
    all_talks_urls.append(get_talk_urls_from_conf(url))


In [4]:
gc_data = pd.DataFrame(columns=['author', 'date', 'text'])
for conf_urls in all_talks_urls[0:20]:
    for url in conf_urls:
        try:
            a_series = pd.Series(get_author_body(url), index = gc_data.columns)
            gc_data = gc_data.append(a_series, ignore_index=True)
        except:
            print(url)


/study/general-conference/2020/10/33video?lang=eng
/general-conference/2016/04/media/4810651550001?lang=eng
/general-conference/2016/04/media/4810668496001?lang=eng
/general-conference/2015/04/media/4135809248001?lang=eng
/general-conference/2015/10/media/4504272728001?lang=eng
/general-conference/2015/10/media/4504246892001?lang=eng
/general-conference/2014/04/media/3434544073001?lang=eng
/general-conference/2014/04/media?lang=eng
/general-conference/2014/04/media?lang=eng
/general-conference/2014/10/media/3810258133001?lang=eng
/general-conference/2013/04/media/2257665547001?lang=eng
/general-conference/2012/04/media/1520978635001?lang=eng
/general-conference/2011/04/media/1515946183001?lang=eng


In [5]:
gc_data.tail()
# test_url = 'https://www.churchofjesuschrist.org/study/general-conference/2020/04/11nelson'

# author, body = get_author_body(test_url)

# conf_url_04_2020 = "/general-conference/2020/04?lang=eng"
# urls_04_2020 = get_talk_urls_from_conf(conf_url_04_2020)
# urls_04_2020




,author,date,url
750,Thomas S. Monson,2011/10,"My brothers and sisters, I know you will agree..."
751,Julie B. Beck,2011/10,It is a privilege to address you at this histo...
752,Silvia H. Allred,2011/10,My husband and I recently visited the city of ...
753,Barbara Thompson,2011/10,"“Lift up thy heart and rejoice, and cleave unt..."
754,Dieter F. Uchtdorf,2011/10,"My dear sisters, what a joy it is to be with y..."


In [24]:
# note: use number of reference as a parameter?? types of references as a parameter?? OT, NT, BoM, other conference talks
# gc_data = pd.DataFrame(columns=['author', 'text'])
# [author, body_block.text]

# to_append = [author, body_block.text]
# a_series = pd.Series(to_append, index = gc_data.columns)
# gc_data = gc_data.append(a_series, ignore_index=True)
# gc_data


,author,text
0,President Russell M. Nelson,"My beloved brothers and sisters, as we welcome..."


In [61]:
# all_talks_urls[:-1]

In [6]:
gc_data.to_csv("/Users/ryanwhetten/Desktop/STAT 426/project/cd_2011_2020_data.csv", index=False)
# all_talks_urls

In [7]:
gc_data.head()


,author,date,url
0,President Russell M. Nelson,2020/04,"My beloved brothers and sisters, as we welcome..."
1,President M. Russell Ballard,2020/04,"Thank you very much, President, for such a won..."
2,Elder James R. Rasband,2020/04,The Book of Mormon Teaches the Doctrine of Chr...
3,Joy D. Jones,2020/04,I’m grateful to focus my remarks today on wome...
4,Elder Neil L. Andersen,2020/04,"Eighteen years after the First Vision, the Pro..."


In [8]:
gc_all_data = pd.DataFrame(columns=['author', 'date', 'text'])
for conf_urls in all_talks_urls:
    for url in conf_urls:
        try:
            a_series = pd.Series(get_author_body(url), index = gc_all_data.columns)
            gc_all_data = gc_all_data.append(a_series, ignore_index=True)
        except:
            print(url)

/study/general-conference/2020/10/33video?lang=eng
/general-conference/2016/04/media/4810651550001?lang=eng
/general-conference/2016/04/media/4810668496001?lang=eng
/general-conference/2015/04/media/4135809248001?lang=eng
/general-conference/2015/10/media/4504272728001?lang=eng
/general-conference/2015/10/media/4504246892001?lang=eng
/general-conference/2014/04/media/3434544073001?lang=eng
/general-conference/2014/04/media?lang=eng
/general-conference/2014/04/media?lang=eng
/general-conference/2014/10/media/3810258133001?lang=eng
/general-conference/2013/04/media/2257665547001?lang=eng
/general-conference/2012/04/media/1520978635001?lang=eng
/general-conference/2011/04/media/1515946183001?lang=eng
/general-conference/2010/04/media/3315334055001?lang=eng
/general-conference/2009/04/media?lang=eng
/general-conference/2009/10/media/1327937417001?lang=eng
/general-conference/2008/04/media/2501794334001?lang=eng
/general-conference/2006/10/media/2678527472001?lang=eng
/general-conference/19

In [12]:
gc_data.to_csv("/Users/ryanwhetten/Desktop/STAT 426/project/cd_all_2020_data.csv", index=False)


In [13]:
gc_data.tail()


,author,date,url
4589,Joseph Anderson,1971/10,I sincerely hope and pray that I may have the ...
4590,Bruce R. McConkie,1971/10,I desire to give some rather plain and affirma...
4591,Thomas S. Monson,1971/10,Yesterday each person assembled in this histor...
4592,Hugh B. Brown,1971/10,One of the compensations for going away from h...
4593,Joseph Fielding Smith,1971/10,My dear brothers and sisters. As we come now t...


In [20]:
christmas_nelson_url = "https://www.churchofjesuschrist.org/broadcasts/article/christmas-devotional/2018/12/four-gifts-that-jesus-christ-offers-to-you?lang=eng"

mysite = urllib.request.urlopen(christmas_nelson_url).read()

soup = BeautifulSoup(mysite)
body = soup.find(id='primary')
        
for ref in body('a'):
        ref.decompose()
        
for p in body.find_all("p"):
    print(p.text)



My beloved brothers and sisters, what a treasured season this is! We love the strains of “Oh, Come, All Ye Faithful” and eagerly come to adore Him: Jesus the Christ—that singular babe of Bethlehem—“the Great Jehovah of the Old Testament [and] the Messiah of the New.”
Tonight let us consider together the blessings that come to us as we focus on the life, mission, doctrine, and Atonement of the Lord Jesus Christ. I invite you, as King Benjamin did for Saints in his day, to “consider on the blessed and happy state of those that keep the commandments of God.” That blessing is for us here and now, but added to it is the eventual promise of “never-ending happiness.” Simply stated, true followers of Jesus Christ have the privilege of experiencing unspeakable joy forever.
I was reminded of this the other day when I met an angel named Lydia. This angel was not dressed in white, and she made it easy for us to visit by coming to my office. Lydia is 12 years old. I was told that she is suffering f

In [2]:
gc_10years_data = pd.read_csv('cd_2011_2020_data.csv')
gc_all_data = pd.read_csv('cd_all_2020_data.csv')


In [9]:
# look at last couple of rows and see how many talks there are
print(gc_10years_data.tail())
print(gc_all_data.tail())


,author,date,text
750,Thomas S. Monson,2011/10,"My brothers and sisters, I know you will agree..."
751,Julie B. Beck,2011/10,It is a privilege to address you at this histo...
752,Silvia H. Allred,2011/10,My husband and I recently visited the city of ...
753,Barbara Thompson,2011/10,"“Lift up thy heart and rejoice, and cleave unt..."
754,Dieter F. Uchtdorf,2011/10,"My dear sisters, what a joy it is to be with y..."
